In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA


In [ ]:
## Read the pdfs from the folder
loader= PyPDFDirectoryLoader("./us_census")
documents=loader.load()
text_splitter= RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_documents=text_splitter.split_documents(documents)
final_documents


In [ ]:
len(final_documents)

In [ ]:
# embedding using hugging face
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5", # or sentence-transformers/all-MinLM-16-v2 
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

In [ ]:
import numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

In [ ]:
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [ ]:
#query using similarity search
query="what is health insurance coverage?"
relevant_docuements=vectorstore.similarity_search(query)
print(relevant_docuements[0].page_content)



In [ ]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=""

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

hf = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    temperature= 0.1, 
    max_new_tokens=500,
    token=""
)

#query = "What is the health insurance coverage?"
print(hf.invoke("What is the health insurance coverage?"))


In [ ]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [ ]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [ ]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)


In [ ]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [ ]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])